In [1]:
# loading and normalizing cifar10
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

import torch.nn as nn
import torch.nn.functional as F

In [2]:

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


In [3]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
                                        
                                        
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [12]:
## creating a function that initializes weights
def init_weight_conv(m):
    if type(m) == nn.Conv2d:
        torch.nn.init.kaiming_normal_(m.weight) #he initialize, can use xavier instead
        m.bias.data.fill_(0.001) # optional bias
        
def init_weight_linear(m):
    if type(m) == nn.Linear:
        torch.nn.init.kaiming_normal_(m.weight)
        m.bias.data.fill_(0.001)
    

In [23]:
# inception style module
class convnet(nn.Module):
    def __init__(self):
        super(convnet, self).__init__()
        self.conv_3 = nn.Sequential(
            nn.Conv2d(3, 8, 1, padding = 0),
            nn.ReLU(),
            nn.Conv2d(8, 8, 3, padding = 1),
            nn.ReLU()
        )
        self.conv_5 = nn.Sequential(
            nn.Conv2d(3, 8, 1, padding = 0),
            nn.ReLU(),
            nn.Conv2d(8, 8, 5, padding = 2),
            nn.ReLU()
        )
        self.max_pool_conv_1 = nn.Sequential(
            nn.MaxPool2d(3, stride = 3), # window size 3x3
            nn.Conv2d(3, 8, 1, padding = 0),
            nn.ReLU()
        )
        self.conv_1 = nn.Sequential(
            nn.Conv2d(3, 8, 1, padding = 0),
            nn.ReLU()
        )
        self.fc1 = nn.Linear(512, 32) # 8*5*5 is flattened 8 filters and 5x5 kernel size
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(32, 10)
        
    def forward(self, x):
        x1 = self.conv_1(x)
        x2 = self.conv_3(x)
        x3 = self.conv_5(x)
        x4 = self.max_pool_conv_1(x)
        x = torch.cat([x1,x2,x3,x4], 1) # concatenate all layers
        x = x.view(x.size(0), -1) # flatten
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
        
net = convnet()
            
            

            

In [24]:
# apply initializers
net.apply(init_weight_conv)
net.apply(init_weight_linear)

convnet(
  (conv_3): Sequential(
    (0): Conv2d(3, 8, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
  )
  (conv_5): Sequential(
    (0): Conv2d(3, 8, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(8, 8, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): ReLU()
  )
  (max_pool_conv_1): Sequential(
    (0): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (1): Conv2d(3, 8, kernel_size=(1, 1), stride=(1, 1))
    (2): ReLU()
  )
  (conv_1): Sequential(
    (0): Conv2d(3, 8, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
  )
  (fc1): Linear(in_features=512, out_features=32, bias=True)
  (dropout): Dropout(p=0.5)
  (fc2): Linear(in_features=32, out_features=10, bias=True)
)

In [25]:
# define loss and optimizer
import torch.optim as optim
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate) #net.parameters

In [26]:
# modified model training to keep track of train/val loss
n_epochs = 2

for epoch in range(n_epochs):
    running_loss = 0.0
    for i, train_data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = train_data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print loss per n minibatches
        running_loss += loss.item()
        if i % 50 == 49:    # print every 50 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0
    
    # keep track of loss in test dataset 
    correct = 0
    total = 0
    running_test_loss = 0.0
    with torch.no_grad():
        for test_data in testloader:
            test_images, test_labels = test_data
            test_outputs = net(test_images)
            test_loss = criterion(test_outputs, test_labels)
            running_test_loss += test_loss.item()
            _, predicted = torch.max(test_outputs.data, 1)
            total += test_labels.size(0)
            correct += (predicted == test_labels).sum().item()


    
    
    # for printing average loss every epoch
    print("===> Epoch {} Complete: Train Avg. Loss: {:.4f}".format(epoch+1, running_loss / len(trainloader)))
    print("===> Epoch {} Complete: Test Avg. Loss: {:.4f}".format(epoch+1, running_test_loss / len(testloader)))
    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
print('Finished Training')

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 1. Got 32 and 10 in dimension 2 at /Users/soumith/code/builder/wheel/pytorch-src/aten/src/TH/generic/THTensorMath.c:3586